**Facultad de Ciencias Exactas, Ingeniería y Agrimensura - UNR**

Tecnicatura Universitaria en Inteligencia Artificial

# Procesamiento del Lenguaje Natural - Trabajo Práctico N°:1.

Integrantes:
- Pace, Bruno. Legajo: P-5295/7.
- Sancho Almenar, Mariano. Legajo:



In [54]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO

## Lectura de DF

Descargamos los dataset desde el link, para no lidiar con inconvenientes al abrir y cerrar sesiones de colab.

In [154]:
!wget -O bgg_database.csv 'https://drive.google.com/uc?id=1yIWOgUV5WyskQvmq48QvF2Lzr0LxpAdq'
!wget -O IMDB-Movie-Data.csv 'https://drive.google.com/uc?id=1YCu3xhZq4C5dYyekiluMabwyWBqQyd2c'

--2024-10-29 14:36:38--  https://drive.google.com/uc?id=1yIWOgUV5WyskQvmq48QvF2Lzr0LxpAdq
Resolving drive.google.com (drive.google.com)... 172.253.117.113, 172.253.117.100, 172.253.117.102, ...
Connecting to drive.google.com (drive.google.com)|172.253.117.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1yIWOgUV5WyskQvmq48QvF2Lzr0LxpAdq [following]
--2024-10-29 14:36:38--  https://drive.usercontent.google.com/download?id=1yIWOgUV5WyskQvmq48QvF2Lzr0LxpAdq
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 173.194.202.132, 2607:f8b0:400e:c00::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|173.194.202.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1833113 (1.7M) [application/octet-stream]
Saving to: ‘bgg_database.csv’

bgg_database.csv    100%[===================>]   1.75M  --.-KB/s    in 0.01s   

2024-10-29 14:36:41 (15

In [155]:
df_juegos: pd.DataFrame = pd.read_csv('bgg_database.csv')
df_peliculas: pd.DataFrame = pd.read_csv('IMDB-Movie-Data.csv')

### Web Scrapping

In [3]:
url: str = 'https://www.gutenberg.org/browse/scores/top1000.php#books-last1'

In [158]:
response: requests.models = requests.get(url)

#extraigo el contenido de la página web
soup: BeautifulSoup = BeautifulSoup(response.text, 'html.parser')

#luego de la inspección, vemos que necesitamos la información de page_content
libro_div = soup.find('div', class_='page_content')

#divido para quedarme con los títulos
paragraphs = libro_div.find_all('a')

In [173]:
#creo lista para poder crear el df
libro: list = [par.text for par in paragraphs]
libro = libro[1::] #mediante slicing salteo la primera linea con información que no nos interesa.

In [174]:
df_libros: pd.DataFrame = pd.DataFrame(libro, columns=['Libro'])

In [175]:
def limpieza_top(libro: str) -> bool:
  """
  Chequea si en string existe el substring 'Top 1000'. Retorna un booleano
  """
  if 'Top 1000' in libro:
    return True
  else:
    return False

In [176]:
df_libros['es_ruido'] = df_libros['Libro'].apply(limpieza_top)

In [177]:
df_libros.drop(df_libros[df_libros['es_ruido'] == True].index, inplace=True)

In [178]:
df_libros = df_libros.drop('es_ruido', axis=1)

In [179]:
df_libros.reset_index(drop=True, inplace=True)

In [180]:
df_libros = df_libros.drop_duplicates()

In [181]:
df_libros.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5988 entries, 0 to 5999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Libro   5988 non-null   object
dtypes: object(1)
memory usage: 93.6+ KB


In [182]:
df_libros.head()

,Libro
0,"Frankenstein; Or, The Modern Prometheus by Mar..."
1,Pride and Prejudice by Jane Austen (2920)
2,The Scarlet Letter by Nathaniel Hawthorne (2562)
3,"Moby Dick; Or, The Whale by Herman Melville (2..."
4,Romeo and Juliet by William Shakespeare (2426)
